In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
from file_loading import file_loader, data_types_converter

file_path = "../dataset/"

df_train = file_loader(file_path + "training/leaf.csv.lz4")
df_test = file_loader(file_path + "testing/leaf.csv.lz4")

In [3]:
numerical_data = ['demand', 'destination_current_public_holiday', 'destination_current_school_holiday', 
                  'destination_days_to_next_public_holiday', 'destination_days_to_next_school_holiday', 
                  'od_destination_time', 'od_number_of_similar_12_hours', 
                  'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours', 'od_origin_month', 
                  'od_origin_time', 'od_origin_week', 'od_origin_weekday', 'od_origin_year', 
                  'od_travel_time_minutes', 'origin_current_public_holiday', 'origin_current_school_holiday', 
                  'origin_days_to_next_public_holiday', 'origin_days_to_next_school_holiday', 
                  'price', 'sale_day', 'sale_day_x', 'sale_month', 'sale_week', 'sale_weekday', 'sale_year']

#### Convertion of data types to float when possible

In [4]:
df_train = data_types_converter(df_train, numerical_data)
df_test = data_types_converter(df_test, numerical_data)

df_train_modified = df_train.copy()
df_test_modified = df_test.copy()

In [5]:
print("train :", df_train_modified.shape)
print("test :", df_test.shape)

train : (632841, 31)
test : (32565, 31)


# Creation of dataset to predict on : 
- evaluation is on the sum of all remaining days until departure

In [6]:
days_to_be_used = [-90, -60, -30, -20, -15, -10, -7, -6, -5, -3, -2, -1]
static_features = ['departure_date',
       'destination_current_public_holiday',
       'destination_current_school_holiday',
       'destination_days_to_next_public_holiday',
       'destination_days_to_next_school_holiday', 'destination_station_name',
       'od_destination_time', 'od_number_of_similar_12_hours',
       'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours',
       'od_origin_month', 'od_origin_time', 'od_origin_week',
       'od_origin_weekday', 'od_origin_year', 'od_travel_time_minutes',
       'origin_current_public_holiday', 'origin_current_school_holiday',
       'origin_days_to_next_public_holiday',
       'origin_days_to_next_school_holiday', 'origin_station_name']#13911 elements if groupby on these
fewer_static_features = ['destination_station_name', 'origin_station_name', 'departure_date', 
                         'od_origin_time', 'od_destination_time']#9173 elements if groupby on these
#Differences in terms of size of groupby results would need further data exploration 

changing_features = ['demand', 'price',
       'sale_date', 'sale_day', 'sale_day_x', 'sale_month', 'sale_week',
       'sale_weekday', 'sale_year']

#### groupby realized to create time series based of same travels:
- number of results changes when more (apparently static) features are used?

In [7]:
from data_pretreatment import splitting_travels, extraction_validation_set

dataset_test = splitting_travels(df_test, fewer_static_features)
# print(len(dataset_test), "different travels in the dataset")
complete_preprocessed_dataset, information_on_travel = extraction_validation_set(dataset_test, days_to_be_used)

100%|████████████████████████████████████████████████████████████████████████████████| 417/417 [00:07<00:00, 57.92it/s]


## First test for prediction --> baseline with random forests

In [8]:
features_names = [#'departure_date',
       'destination_current_public_holiday',
       'destination_current_school_holiday',
       'destination_days_to_next_public_holiday',
       'destination_days_to_next_school_holiday',# 'destination_station_name',
       'od_destination_time', 'od_number_of_similar_12_hours',
       'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours',
       'od_origin_month', 'od_origin_time', 'od_origin_week',
       'od_origin_weekday', 'od_origin_year', 'od_travel_time_minutes',
       'origin_current_public_holiday', 'origin_current_school_holiday',
       'origin_days_to_next_public_holiday',
       'origin_days_to_next_school_holiday', 'price', #'origin_station_name', #'sale_date', 
       'sale_day', 'sale_day_x', 'sale_month', 'sale_week',
       'sale_weekday', 'sale_year']

target_train = df_train_modified.demand
features_train = df_train_modified[features_names]

from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, max_depth=6)#, random_state=27)
regr.fit(features_train, target_train)

RandomForestRegressor(max_depth=6)

#### Prediction of each day separately

In [9]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values = df_test_modified.demand
features_test = df_test_modified[features_names]
prediction_all_days = regr.predict(features_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_all_days))
print("mse :", mean_squared_error(real_values, prediction_all_days))
print("r2 :", r2_score(real_values, prediction_all_days))

print("std mae :", np.std(np.abs(real_values - prediction_all_days)))

mae : 1.9420217021261619
mse : 20.164198690441026
r2 : 0.6657907695391516
std mae : 4.048796166629468


#### Prediction on the sum of remaining days

In [10]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values, prediction_until_travel, information_on_travel_corrected = protocol_classical_ML_predictor(complete_preprocessed_dataset, 
                                                                                                        features_names, regr, 
                                                                                                        information_on_travel)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_until_travel))
print("mse :", mean_squared_error(real_values, prediction_until_travel))
print("r2 :", r2_score(real_values, prediction_until_travel))

print("std mae :", np.std(np.abs(np.array(real_values) - np.array(prediction_until_travel))))

mae : 35.68107989464443
mse : 4834.850087796312
r2 : 0.8249993695389566
std mae : 59.680068912060705


### First improvement : categorical features preprocessing

In [11]:
from data_pretreatment import encode_and_bind

cat_features = ['destination_station_name', 'origin_station_name']

for feature in cat_features:
    df_train_modified = encode_and_bind(df_train_modified, feature)
    df_test_modified = encode_and_bind(df_test_modified, feature)

df_train, df_test = df_train.align(df_test, join='left', axis=1)  # inner join

### Second improvement : add of a missing parameter

In [12]:
df_train_modified['od_origin_day'] = pd.to_datetime(df_train_modified.departure_date).dt.day
df_test_modified['od_origin_day'] = pd.to_datetime(df_test_modified.departure_date).dt.day

### Thrird improvement : convertion of cyclic features to adapted ones 

In [13]:
from data_pretreatment import cyclic_features_transform

periodic_features_day = ['od_origin_day', 'sale_day'] 
periodic_features_month = ['od_origin_month', 'sale_month']

df_train_modified = cyclic_features_transform(df_train_modified, periodic_features_day, periodic_features_month)
df_test_modified = cyclic_features_transform(df_test_modified, periodic_features_day, periodic_features_month)

## Second test with transformed features

In [14]:
features_names = [
       'destination_current_public_holiday',
       'destination_current_school_holiday',
       'destination_days_to_next_public_holiday',
       'destination_days_to_next_school_holiday',
       'od_destination_time', 'od_number_of_similar_12_hours',
       'od_number_of_similar_2_hours', 'od_number_of_similar_4_hours',
       'od_origin_month', 'od_origin_time', 'od_origin_week',
       'od_origin_weekday', 'od_origin_year', 'od_travel_time_minutes',
       'origin_current_public_holiday', 'origin_current_school_holiday',
       'origin_days_to_next_public_holiday',
       'origin_days_to_next_school_holiday', 'price', 
       'sale_day', 'sale_day_x', 'sale_month', 'sale_week',
       'sale_weekday', 'sale_year']

target_train = df_train_modified.demand
features_train = df_train_modified[features_names]

from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, max_depth=6)#, random_state=27)
regr.fit(features_train, target_train)

RandomForestRegressor(max_depth=6)

#### Each day separetely

In [15]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values = df_test_modified.demand
features_test = df_test_modified[features_names]
prediction_all_days = regr.predict(features_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_all_days))
print("mse :", mean_squared_error(real_values, prediction_all_days))
print("r2 :", r2_score(real_values, prediction_all_days))

print("std mae :", np.std(np.abs(real_values - prediction_all_days)))

mae : 1.9416716920485655
mse : 20.12559822764714
r2 : 0.6664305485437045
std mae : 4.044194514108561


#### Sum of all remaining days

In [16]:
from classical_ml_evaluation import protocol_classical_ML_predictor

real_values, prediction_until_travel, information_on_travel_corrected = protocol_classical_ML_predictor(complete_preprocessed_dataset, 
                                                                                                        features_names, regr, information_on_travel)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

print("mae :", mean_absolute_error(real_values, prediction_until_travel))
print("mse :", mean_squared_error(real_values, prediction_until_travel))
print("r2 :", r2_score(real_values, prediction_until_travel))

print("std mae :", np.std(np.abs(np.array(real_values) - np.array(prediction_until_travel))))

mae : 35.63564530289728
mse : 4829.511413520632
r2 : 0.8251926064226458
std mae : 59.6624856787473


#### --> No real improvement with features engineering
Final results with classical ML

In [17]:
len(information_on_travel_corrected)

4556

### Third test : xgboost better? --> No

In [18]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

params = {
    # Parameters that we are going to tune.
    'max_depth': 3,
    'min_child_weight': 1,
    'eta':0.01,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric': 'mae'
}
xgtrain = xgb.DMatrix(np.array(X_train), np.array(y_train))
xgval = xgb.DMatrix(np.array(X_val), np.array(y_val))

model = xgb.train(
    params,
    xgtrain,
    num_boost_round=1000,
    evals=[(xgval, "Validation")],
    early_stopping_rounds=10,
    verbose_eval=False
)
best_iteration = model.best_ntree_limit

real_values, prediction_until_travel, information_on_travel_corrected = protocol_classical_ML_predictor(
    complete_preprocessed_dataset, features_names, regr, information_on_travel)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("mae :", mean_absolute_error(real_values, prediction_until_travel))
print("mse :", mean_squared_error(real_values, prediction_until_travel))
print("r2 :", r2_score(real_values, prediction_until_travel))

# print("with split :", mean_absolute_error(model.predict(xgtest, ntree_limit=best_iteration), target_test))

mae : 35.63564530289728
mse : 4829.511413520632
r2 : 0.8251926064226458


# Deep Learning Model

In [19]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Proposition is to only consider the prices of the last 14 days as inputs for this LSTM, and the demand of the last day as target
==> A more complex approach would require more time

### Normalization of features : necessary for LSTM

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler_demand = MinMaxScaler(feature_range=(-1, 1))
df_train["scaled_demand"] = scaler_demand.fit_transform(np.array(df_train.demand).reshape(-1, 1))
scaler_price = MinMaxScaler(feature_range=(-1, 1))
df_train["scaled_price"] = scaler_price.fit_transform(np.array(df_train.price).reshape(-1, 1))

### time series separation into multiple dataframes

In [21]:
from data_pretreatment import splitting_travels, extraction_validation_set

dataset_train = splitting_travels(df_train, fewer_static_features)

In [22]:
train_window = 14

In [ ]:
from data_pretreatment import create_inout_sequences
train_inout_seq = create_inout_sequences(dataset_train, train_window)

 98%|████████████████████████████████████████████████████████████████████████████▏ | 8959/9173 [02:20<00:03, 61.41it/s]

In [ ]:
list_of_tensors = [(torch.FloatTensor(np.array(df[0])).view(-1), torch.FloatTensor(np.array(df[1])).view(-1)) for df in train_inout_seq]
# X_train_data_normalized = torch.stack(list_of_tensors)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1, self.hidden_layer_size),
                            torch.zeros(1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [ ]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
print(model)

In [ ]:
import random

epochs = 100
batch_size = 200

for i in range(epochs):
    batch = random.choices(list_of_tensors, k=batch_size)
    for k in range(4):
    
        for seq, labels in batch:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))

            y_pred = model(seq)

            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()
        
        print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

    #     if i%25 == 1:
        

print(f'epoch: {i:3} loss: {single_loss.item():10.12f}')

#### Adaptation of the validation dataset to LSTM constraints

In [ ]:
from data_pretreatment import create_inout_sequences_validation
all_validation_data = []; all_validation_target = []
for ts in tqdm(complete_preprocessed_dataset):
    available_part, unavailable_part = ts
    if len(available_part) > train_window: #prediction possible
        limit = len(available_part)
        whole_df = pd.concat([available_part.iloc[len(available_part)-train_window:], unavailable_part])
        features_val, target_val = create_inout_sequences_validation(whole_df, train_window)
        all_validation_data.extend(features_val)
        all_validation_target.extend(target_val)

In [ ]:
list_of_tensors_test = [(torch.FloatTensor(np.array(df)).view(-1)) for df in all_validation_data]

### Predictions on validation data --> test on single demand's day each time (no sum until departure)

In [ ]:
model.eval()
test_inputs = []
kept_targets = []
for i in tqdm(range(len(list_of_tensors_test))):
    seq = torch.FloatTensor(list_of_tensors_test[i])
    if len(seq)>0:
        with torch.no_grad():
            model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))
            test_inputs.append(model(seq).item())
            kept_targets.append(all_validation_target[i])

#### To inverse previous scaling

In [ ]:
actual_predictions = scaler_demand.inverse_transform(np.array(test_inputs).reshape(-1, 1))
print(actual_predictions.shape)
print(np.array(kept_targets).shape)

#### Results

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("mae :", mean_absolute_error(kept_targets, actual_predictions))
print("mse :", mean_squared_error(kept_targets, actual_predictions))
print("r2 :", r2_score(kept_targets, actual_predictions))

### Conclusion on deep learning approach : 
- does not work well, no convergence (seen during the training), then performances remain bad with validation